In [ ]:
%pip install nltk 

In [2]:
%pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f1/7d/2e562207176a5dcdad513085670674bb11ffaf37e1393eacb6d7fb502481/scikit_learn-1.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy<2.0,>=1.17.3 from https://files.pythonhosted.org/packages/93/fd/3f826c6d15d3bdcf65b8031e4835c52b7d9c45add25efa2314b53850e1a2/numpy-1.26.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
     -------------------------- ------------- 41.0/61.1 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.1 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 61.1/61.1 kB 651.8 kB/s eta 0:00:00
  Obtaining dependency information for scip

In [7]:
# nltk.download('stopwords') # ----------- python -m nltk.downloader stopwords
# nltk.download('punkt') # --------------- python -m nltk.downloader punkt
# nltk.download('wordnet') # ------------- python -m nltk.downloader wordnet

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity      
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# import spacy
lemmatizer = nltk.stem.WordNetLemmatizer()
# Download required NLTK data
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# import streamlit

In [3]:
data = pd.read_csv("dataset\Samsung Dialog.txt", na_filter=False, sep = ':', header = None)
# df = df[['Sales Agent', 'Customer']]
data

,0,1
0,Customer,"Hi, I'm looking to buy a new phone, and I'm i..."
1,Sales Agent,"Great, we have a wide range of Samsung phones..."
2,Customer,"Well, I want a phone with a good camera, long..."
3,Sales Agent,Absolutely. We have a lot of great options th...
4,Customer,"No, I haven't. Tell me more about it."
5,Sales Agent,The Galaxy S21 Ultra has a 108-megapixel came...
6,Customer,That sounds great. How much does it cost?
7,Sales Agent,"The Galaxy S21 Ultra starts at $1,199, but we..."
8,Customer,"Okay, I'm interested. But I have a few more q..."
9,Sales Agent,The Galaxy S21 Ultra comes with a standard on...


In [5]:
customer_data = data[data[0] == 'Customer'].reset_index(drop=True)
sales_agent_data = data[data[0] == 'Sales Agent'].reset_index(drop=True)

# Rename columns for clarity
customer_data = customer_data.rename(columns={1: "Customer", 0: 'drop1'})
sales_agent_data = sales_agent_data.rename(columns={1: "Sales Agent", 0: 'drop2'})

# Combine the two DataFrames
result_data = pd.concat([customer_data, sales_agent_data], axis=1)
result_data.drop(['drop1', 'drop2'], axis = 1, inplace = True)
result_data

,Customer,Sales Agent
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones..."
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we..."
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...
5,That's good to know. And what about the opera...,"Yes, the Galaxy S21 Ultra runs on Android 11,..."
6,"Okay, that's good. But I'm also interested in...",Absolutely. The Galaxy A52 is a great mid-ran...
7,That sounds like a good option for me. How mu...,"The Galaxy A52 starts at $399, but again, we ..."
8,"Okay, I'll think about it. But can you also t...",Of course. The Galaxy Z Fold2 is a really uni...
9,"That sounds really cool, but it also sounds e...","The Galaxy Z Fold2 starts at $1,999, but agai..."


In [6]:
# Define a function for text preprocessing (including lemmatization)
def preprocess_text(text):
    
    # Identifies all sentences in the result_data
    sentences = nltk.sent_tokenize(text)
    
    # Tokenize and lemmatize each word in each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        # Turns to basic root - each word in the tokenized word found in the tokenized sentence - if they are all alphanumeric 
        # The code above does the following:
        # Identifies every word in the sentence 
        # Turns it to a lower case 
        # Lemmatizes it if the word is alphanumeric

        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)
    
    return ' '.join(preprocessed_sentences)


result_data['tokenized Questions'] = result_data['Customer'].apply(preprocess_text)
result_data.head()

,Customer,Sales Agent,tokenized Questions
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones...",hi i looking to buy a new phone and i interest...
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...,well i want a phone with a good camera long ba...
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...,no i have tell me more about it
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we...",that sound great how much doe it cost
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...,okay i interested but i have a few more questi...


In [7]:
# Create a corpus by flattening the preprocessed questions
corpus = result_data['tokenized Questions'].tolist()

In [8]:
# Vectorize corpus
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(corpus)
# TDIDF is a numerical statistic used to evaluate how important a word is to a document in a collection or corpus. 
# The TfidfVectorizer calculates the Tfidf values for each word in the corpus and uses them to create a matrix where each row represents a document and each column represents a word. 
# The cell values in the matrix correspond to the importance of each word in each document.

In [10]:
def get_response(user_input):
    global most_similar_index
    
    user_input_processed = preprocess_text(user_input) # ....................... Preprocess the user's input using the preprocess_text function

    user_input_vector = tfidf_vectorizer.transform([user_input_processed])# .... Vectorize the preprocessed user input using the TF-IDF vectorizer

    similarity_scores = cosine_similarity(user_input_vector, X) # .. Calculate the score of similarity between the user input vector and the corpus (df) vector

    most_similar_index = similarity_scores.argmax() # ..... Find the index of the most similar question in the corpus (df) based on cosine similarity

    return result_data['Sales Agent'].iloc[most_similar_index] # ... Retrieve the corresponding answer from the df DataFrame and return it as the chatbot's response

# create greeting list 
greetings = ["Hey There.... I am a creation of Teelash_K.... How can I help",
            "Hi Human.... How can I help",
            "Good Day .... How can I help", 
            "Hello There... How can I be useful to you today", "Hello! How can I start your week off right?",
            "Good evening, What's on your mind today?", "Welcome back! How can I assist you today?"]
            

exits = ['thanks bye', 'bye', 'quit', 'exit', 'bye bye', 'close']
farewell = ['Thanks....see you soon', 'Bye, See you soon', 'Bye... See you later', 'Bye... come back soon']

random_farewell = random.choice(farewell) # ---------------- Randomly select a farewell message from the list
random_greetings = random.choice(greetings) # -------- Randomly select greeting message from the list

# Test your chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in exits:
        print(f"\nChatbot: {random_farewell}!")
        break
    if user_input.lower() in ['hi', 'hello', 'hey', 'hi there']:
        print(f"\nChatbot: {random_greetings}!")
    else:   
        response = get_response(user_input)
        print(f"\nChatbot: {response}")


Chatbot: Good evening, What's on your mind today?!

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot:  Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?

Chatbot:  Sure, the Galaxy A72 is another great mid-range phone that has a lot of the same features as the A52, but with a bigger display, bigger battery, and more cameras. It has a 6.7-inch Super AMOLED display, a 5,000mAh battery, and four cameras, including a 64-megapixel main camera, a 12-megapixel ultra-wide camera, an 8-megapixel telephoto came

MODEL TECHNIQUE

In [11]:
tfidf_vectorizer = TfidfVectorizer()
xtrain = tfidf_vectorizer.fit_transform(result_data['tokenized Questions'])
# Xtrain is the preprocessed questions 

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Transform the Y 
result_data['Answers_ID'] = le.fit_transform(result_data['Sales Agent'])
result_data.head()

ytrain = result_data['Answers_ID'].values
# ytrain is the transformed Answers 

In [19]:
result_data.head()

,Customer,Sales Agent,tokenized Questions,Answers_ID
0,"Hi, I'm looking to buy a new phone, and I'm i...","Great, we have a wide range of Samsung phones...",hi i looking to buy a new phone and i interest...,3
1,"Well, I want a phone with a good camera, long...",Absolutely. We have a lot of great options th...,well i want a phone with a good camera long ba...,1
2,"No, I haven't. Tell me more about it.",The Galaxy S21 Ultra has a 108-megapixel came...,no i have tell me more about it,11
3,That sounds great. How much does it cost?,"The Galaxy S21 Ultra starts at $1,199, but we...",that sound great how much doe it cost,12
4,"Okay, I'm interested. But I have a few more q...",The Galaxy S21 Ultra comes with a standard on...,okay i interested but i have a few more questi...,10


In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

mnb = MultinomialNB()
mnb.fit(xtrain, ytrain)

rf = RandomForestClassifier()
rf.fit(xtrain, ytrain)

train_predict = mnb.predict(xtrain)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(train_predict, ytrain))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1

    accuracy              

In [21]:
def get_response(user_input):
    global results
    user_input_processed = preprocess_text(user_input) # ....................... Preprocess the user's input using the preprocess_text function

    user_input_vector = tfidf_vectorizer.transform([user_input_processed])# .... Vectorize the preprocessed user input using the TF-IDF vectorizer

    results = mnb.predict(user_input_vector)

    for elem in results:
        row_df = result_data.loc[result_data.isin([elem]).any(axis=1)]
        print(row_df['Sales Agent'].values)

# create greeting list 
greetings = ["Hey There.... I am a creation of Teelash_K.... How can I help",
            "Hi Human.... How can I help",
            "Good Day .... How can I help", 
            "Hello There... How can I be useful to you today", "Hello! How can I start your week off right?",
            "Good evening, What's on your mind today?", "Welcome back! How can I assist you today?"]
            

exits = ['thanks bye', 'bye', 'quit', 'exit', 'bye bye', 'close']
farewell = ['Thanks....see you soon', 'Bye, See you soon', 'Bye... See you later', 'Bye... come back soon']

random_farewell = random.choice(farewell) # ---------------- Randomly select a farewell message from the list
random_greetings = random.choice(greetings) # -------- Randomly select greeting message from the list

# Test your chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in exits:
        print(f"\nChatbot: {random_farewell}!")
        break
    if user_input.lower() in ['hi', 'hello', 'hey', 'hi there']:
        print(f"\nChatbot: {random_greetings}!")
    else:   
        response = get_response(user_input)
        print(f"\nChatbot: {response}")


Chatbot: Welcome back! How can I assist you today?!

Chatbot: Welcome back! How can I assist you today?!

Chatbot: Welcome back! How can I assist you today?!
[' Great, we have a wide range of Samsung phones to choose from. What features are you looking for in a phone?']

Chatbot: None
[" You're welcome. If you have any other questions or concerns, don't hesitate to reach out to us. We're always here to help."]

Chatbot: None
[" You're welcome. If you have any other questions or concerns, don't hesitate to reach out to us. We're always here to help."]

Chatbot: None

Chatbot: Bye... come back soon!
